# New evaluation codes

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from os.path import exists

sys.path.append('../..')

In [3]:
from stable_baselines3 import DQN

In [4]:
from experiments import preset_qcb_small
alpha = 0.191500954
beta = 0.030798858
extract = False
params, max_peaks = preset_qcb_small(None, alpha=alpha, beta=beta, extract_chromatograms=extract)
params, max_peaks

2022-08-23 10:00:32.582 | INFO     | experiments:get_samplers:269 - Loaded /Users/joewandy/Work/git/vimms-gym/pickles/samplers_QCB_small_gaussian.p


({'chemical_creator': {'mz_range': (100, 110),
   'rt_range': (400, 500),
   'intensity_range': (10000.0, 1e+20),
   'n_chemicals': (20, 50),
   'mz_sampler': <vimms.ChemicalSamplers.MZMLFormulaSampler at 0x7f9ca4abe760>,
   'ri_sampler': <vimms.ChemicalSamplers.MZMLRTandIntensitySampler at 0x7f9ca64ccd60>,
   'cr_sampler': <vimms.ChemicalSamplers.GaussianChromatogramSampler at 0x7f9ca64ee700>},
  'noise': {'enable_spike_noise': True,
   'noise_density': 0.1,
   'noise_max_val': 1000.0,
   'mz_range': (100, 110)},
  'env': {'ionisation_mode': 'Positive',
   'rt_range': (400, 500),
   'isolation_window': 0.7,
   'mz_tol': 10,
   'rt_tol': 120,
   'alpha': 0.191500954,
   'beta': 0.030798858}},
 100)

In [5]:
from vimms_gym.env import DDAEnv
from stable_baselines3.common.monitor import Monitor
from vimms.Evaluation import EvaluationData
from vimms_gym.common import EVAL_METRIC_REWARD
from vimms_gym.evaluation import evaluate
from tune import TrialEvalCallback

import numpy as np
from timeit import default_timer as timer
from datetime import timedelta

In [6]:
custom_objects = {
    "learning_rate": 0.0,
    "lr_schedule": lambda _: 0.0,
    "clip_range": lambda _: 0.0,
}    

fname = os.path.join('..', 'DQN', 'DQN_9.zip') 
model = DQN.load(fname, custom_objects=custom_objects)

/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:561: UserWarning:

This system does not have apparently enough memory to store the complete replay buffer 9.87GB > 0.34GB

/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/dqn/dqn.py:152: UserWarning:

The number of environments used is greater than the target network update interval (20 > 1), therefore the target network will be updated after each call to env.step() which corresponds to 20 steps.



In [7]:
eval_env = DDAEnv(max_peaks, params)
eval_env

In [8]:
eval_env.env_params, eval_env.alpha, eval_env.beta

({'ionisation_mode': 'Positive',
  'rt_range': (400, 500),
  'isolation_window': 0.7,
  'mz_tol': 10,
  'rt_tol': 120,
  'alpha': 0.191500954,
  'beta': 0.030798858},
 0.191500954,
 0.030798858)

In [9]:
eval_env = Monitor(eval_env)
eval_env

<Monitor<DDAEnv instance>>

In [10]:
eval_metric = EVAL_METRIC_REWARD
eval_callback = TrialEvalCallback(eval_env, None, eval_metric)

In [11]:
n_eval_episodes = 10
verbose = 2

In [12]:
env = eval_callback.eval_env
env

In [13]:
deterministic = eval_callback.deterministic
deterministic = False
deterministic

False

In [14]:
episode_count = 0
episode_count_target = n_eval_episodes
current_reward = 0
current_length = 0
observations = env.reset()
states = None
episode_starts = np.ones((env.num_envs,), dtype=bool)
episode_starts

array([ True])

In [15]:
episode_rewards = []
episode_eval_results = []
episode_lengths = []
start = timer()
while episode_count < episode_count_target:
    actions, states = model.predict(observations, state=states,
                                    episode_start=episode_starts,
                                    deterministic=deterministic)
    observations, rewards, dones, infos = env.step(actions)
    # print(rewards, current_reward, current_length, dones)
    episode_starts = dones
    current_reward += rewards[0]
    current_length += 1

    if dones[0]:  # when done, episode would be reset automatically
        val = current_reward
        eval_res = evaluate(eval_data, format_output=False)
        episode_eval_results.append(eval_res)
        if verbose > 0:
            end = timer()
            print('Evaluation episode %d finished: metric %f, timedelta=%s' % (
                episode_count, val, str(timedelta(seconds=end - start))))
            start = timer()
        episode_rewards.append(val)
        episode_lengths.append(current_length)
        episode_count += 1
        current_reward = 0
        current_length = 0

    # store previous results for evaluation before 'done'
    # this needs to be here, because VecEnv is automatically reset when done
    inner_env = env.envs[0].env
    eval_data = EvaluationData(inner_env.vimms_env)


Evaluation episode 0 finished: metric -468.698419, timedelta=0:00:00.325830
Evaluation episode 1 finished: metric -475.066150, timedelta=0:00:00.310465
Evaluation episode 2 finished: metric -465.754302, timedelta=0:00:00.398238
Evaluation episode 3 finished: metric -490.301367, timedelta=0:00:00.237729
Evaluation episode 4 finished: metric -473.511853, timedelta=0:00:00.397659
Evaluation episode 5 finished: metric -478.482564, timedelta=0:00:00.454557
Evaluation episode 6 finished: metric -471.573760, timedelta=0:00:00.332286
Evaluation episode 7 finished: metric -476.219832, timedelta=0:00:00.299827
Evaluation episode 8 finished: metric -472.525067, timedelta=0:00:00.286736
Evaluation episode 9 finished: metric -479.003029, timedelta=0:00:00.290128


In [16]:
episode_rewards

[-468.69841850176454,
 -475.0661503598094,
 -465.75430223718286,
 -490.30136651545763,
 -473.51185309141874,
 -478.4825639612973,
 -471.5737603493035,
 -476.2198322825134,
 -472.5250670388341,
 -479.0030293315649]

In [17]:
np.mean(episode_rewards), np.std(episode_rewards)

(-475.11363436691465, 6.394680608259049)

In [18]:
metric = [e['coverage_prop'] for e in episode_eval_results]
np.mean(metric), np.std(metric)

(0.6422401820265958, 0.15266011958640682)

In [19]:
metric = [e['intensity_prop'] for e in episode_eval_results]
np.mean(metric), np.std(metric)

(0.3849329415507192, 0.14393627435839754)

In [20]:
metric = [e['f1'] for e in episode_eval_results]
np.mean(metric), np.std(metric)

(0.15115538930649694, 0.06713096564078858)